In [51]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
import math
import importlib
import Attention_models
importlib.reload(Attention_models)
from Attention_models import ShallowAttentionNet
from shallow_laurits import ShallowFBCSPNet
from torchinfo import summary

In [58]:
in_chans = 22
n_classes = 4
input_window_samples = 1125
model = ShallowAttentionNet(in_chans,n_classes,input_window_samples,mini_pool_size=2)
ref_model = ShallowFBCSPNet(in_chans,n_classes,input_window_samples)

In [53]:
summary(model,input_size=(1, 22, 1125),col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowAttentionNet                      [1, 22, 1125]             [1, 4]                    --                        --
├─Conv2d: 1-1                            [1, 1, 22, 1125]          [1, 10, 22, 1101]         260                       [1, 25]
├─SpatialAttention: 1-2                  [1, 10, 22, 1101]         [1, 10, 22, 1101]         --                        --
│    └─Conv1d: 2-1                       [10, 22, 1101]            [10, 22, 1101]            506                       [1]
│    └─Conv1d: 2-2                       [10, 22, 1101]            [10, 22, 1101]            506                       [1]
│    └─Conv1d: 2-3                       [10, 22, 1101]            [10, 22, 1101]            506                       [1]
├─BatchNorm2d: 1-3                       [1, 10, 22, 1101]         [1, 10, 22, 1101]         20                        --
├─AvgP

In [54]:
summary(ref_model,input_size=(1, 22, 1125),col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet                          [1, 22, 1125]             [1, 4]                    --                        --
├─Conv2d: 1-1                            [1, 1, 22, 1125]          [1, 40, 22, 1101]         1,040                     [1, 25]
├─Conv2d: 1-2                            [1, 40, 22, 1101]         [1, 40, 1, 1101]          35,240                    [22, 1]
├─BatchNorm2d: 1-3                       [1, 40, 1, 1101]          [1, 40, 1, 1101]          80                        --
├─AvgPool2d: 1-4                         [1, 40, 1, 1101]          [1, 40, 1, 11]            --                        [1, 100]
├─Dropout: 1-5                           [1, 440]                  [1, 440]                  --                        --
├─Linear: 1-6                            [1, 440]                  [1, 4]                    1,764                     -

In [55]:
import pickle
with open('Datasets/test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)

X = torch.stack([torch.from_numpy(test_set[i][0]) for i in range(len(test_set))])

In [56]:

X_un = X.unsqueeze(dim=1)
print(X_un.shape)

torch.Size([2592, 1, 22, 1125])


In [57]:
model.eval()
out = model(X[:1])
print(out.shape)
print(out)

torch.Size([1, 4])
tensor([[ 0.0170, -0.0066,  0.0045,  0.0245]], grad_fn=<AddmmBackward0>)
